Code adapted from NVIDIA OpenACC example: laplace2d-kernels.c

Copyright (c) 2012, NVIDIA CORPORATION. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:
 * Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
 * Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
 * Neither the name of NVIDIA CORPORATION nor the names of its
   contributors may be used to endorse or promote products derived
   from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT OWNER OR
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
%load_ext line_profiler

In [2]:
%load_ext snakeviz

In [3]:
import numpy as np
import time

In [4]:
import pickle

In [21]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:-1, 1:-1] = 0.25 * (A[1:-1, 2:] + A[1:-1, :-2] + A[2:, 1:-1] + A[:-2, 1:-1])      
        diff = np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [22]:
res = np_laplace(12, 12)

In [23]:
%lprun -f np_laplace np_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 9.027721405029297 s



In [ ]:
pow_max = 6
N = [2 ** val for val in range(2, pow_max)]

In [ ]:
N

In [ ]:
np_res = {}
for n in N:
    res = %timeit -o -n 3 np_laplace(n, n)
    np_res[n] = res
pickle.dump(np_res, open('np_times.p', 'wb'))

In [ ]:
np_res

# Adding numba jit decorator

In [8]:
from numba import jit, njit

In [9]:
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def update_a_jit(A, Anew):
    error = 0.0
    n, m = A.shape

    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
            error = max(error, abs(Anew[j, i] - A[j, i]))

    return Anew, error

In [10]:
# this did not make much difference
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def a_to_anew_jit(A, Anew):
    error = 0.0
    n, m = A.shape
    
    for j in range(1, n-1):
        for i in range(1, m-1):
            A[j, i] = Anew[j, i]
    
    return A

In [14]:
def jit_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        Anew, error = update_a_jit(A, Anew)
        # A[:] = Anew[:]
        A = a_to_anew_jit(A, Anew)  # this was essentially the same as the line above

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [15]:
res = jit_laplace(512, 512) # once to compile

In [16]:
%lprun -f jit_laplace jit_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.4284555912017822 s



In [ ]:
jit_res = {}
for n in N:
    res = %timeit -o -n 3 jit_laplace(n, n)
    jit_res[n] = res
pickle.dump(jit_res, open('jit_times.p', 'wb'))

# Implementing `guvectorize`

In [17]:
from numba import guvectorize

In [ ]:
from math import fabs

### CPU Vectorization

In [ ]:
@guvectorize(['float32[:,:], float32[:,:]',
              'float64[:,:], float64[:,:]'],
             '(n,m)->(n,m)', nopython=True,
             target='cpu')
def update_a_serial0(A, Anew):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

In [51]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='cpu')
def update_a_serial(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [43]:
A = np.zeros((4,4))
A[:, [0, -1]] = 1.0
Anew = np.zeros_like(A)
error = np.zeros(1)
Anew, error = update_a_serial(A, Anew, error)
np.max(np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])), error[0]

(0.25, 0.25)

In [52]:
def cpu_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_serial(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A

In [53]:
res = cpu_laplace(512, 512)

Jacobi relaxation Calculation: 512 x 512 mesh

 total: 1.2527709007263184 s



In [49]:
%lprun -f cpu_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.5279054641723633 s



In [60]:
%timeit cpu_laplace(1024, 1024)

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 8.300802230834961 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.532527685165405 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.731838464736938 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.843186378479004 s

1 loop, best of 3: 9.54 s per loop


In [ ]:
cpu_res = {}
for n in N:
    res = %timeit -o -n 3 cpu_laplace(n, n)
    cpu_res[n] = res
pickle.dump(cpu_res, open('cpu_times.p', 'wb'))

### parallel CPU vectorization

In [55]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='parallel')
def update_a_parallel(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [56]:
def par_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_parallel(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A

In [57]:
res = par_laplace(512, 512, debug=False)

Jacobi relaxation Calculation: 512 x 512 mesh

 total: 2.017565965652466 s



In [58]:
%lprun -f par_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.238203763961792 s



In [61]:
%timeit par_laplace(1024, 1024)

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 11.581105470657349 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 11.712381362915039 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 12.052513837814331 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 12.310455560684204 s

1 loop, best of 3: 11.7 s per loop


In [ ]:
par_res = {}
for n in N:
    res = %timeit -o -n 3 par_laplace(n, n)
    par_res[n] = res
pickle.dump(par_res, open('par_times.p', 'wb'))

### CUDA vectorization

In [55]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='cuda')
def update_a_cuda(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [56]:
def cuda_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_cuda(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A

In [ ]:
res = cuda_laplace(512, 512, debug=False)

In [ ]:
%lprun -f cuda_laplace cuda_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

In [ ]:
cuda_res = {}
for n in N:
    res = %timeit -o -n 3 cuda_laplace(n, n)
    cuda_res[n] = res
pickle.dump(cuda_res, open('cuda_times.p', 'wb'))